# Carga de datos 


## Librerias 

In [1]:

# Importación de las funciones creadas en nuestro archivo de soporte
# -----------------------------------------------------------------------
import sys
sys.path.append("..")

# from src import soporte_carga as sc
from src import soporte_queries_creacion as sqc


# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd
# Para trabajar con postgresql
# -----------------------------------------------------------------------
import psycopg2

# Para trabajar gestionar los nulos
# -----------------------------------------------------------------------
import numpy as np

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

## Creacionde base de datos

In [2]:
def establecer_conn(database_name, postgres_pass, usuario, host="localhost"):
    """
    Establece una conexión a una base de datos de PostgreSQL.

    Params:
        - database_name (str): El nombre de la base de datos a la que conectarse.
        - postgres_pass (str): La contraseña del usuario de PostgreSQL.
        - usuario (str): El nombre del usuario de PostgreSQL.
        - host (str, opcional): La dirección del servidor PostgreSQL. Por defecto es "localhost".

    Returns:
        psycopg2.extensions.connection: La conexión establecida a la base de datos PostgreSQL.

    """

    # Crear la conexión a la base de datos PostgreSQL
    conn = psycopg2.connect(
        host=host,
        user=usuario,
        password=postgres_pass,
        database=database_name
    )

    # Establecer la conexión en modo autocommit
    conn.autocommit = True # No hace necesario el uso del commit al final de cada sentencia de insert, delete, etc.
    
    return conn



# conexion a postgres
conn = establecer_conn("postgres", "admin", "my_user") # Nos conectamos a la base de datos de postgres por defecto para poder crear la nueva base de datos

# creamos un cursor con la conexion que acabamos de crear
cur = conn.cursor()


In [3]:
cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", ("sanidad42",)) 

In [4]:

# Almacenamos en una variable el resultado del fetchone. Si existe tendrá una fila sino será None
bbdd_existe = cur.fetchone() # recordamos que fetchone nos devuelve unico valor  
type(bbdd_existe)

tuple

In [5]:
# Si bbdd_existe es None, creamos la base de datos
if not bbdd_existe:
    cur.execute(f'CREATE DATABASE {"sanidad42"};')
    print(f'Base de datos {"sanidad42"} creada con éxito')
else:
    print(f"La base de datos ya existe")
    

La base de datos ya existe


In [6]:
# Cerramos el cursor y la conexion
cur.close()
conn.close()

In [7]:
def crear_db(database_name):
    # conexion a postgres
    conn = establecer_conn("postgres", "admin", "postgres") # Nos conectamos a la base de datos de postgres por defecto para poder crear la nueva base de datos
    
    # creamos un cursor con la conexion que acabamos de crear
    cur = conn.cursor()
    
    cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (database_name,))
    
    # Almacenamos en una variable el resultado del fetchone. Si existe tendrá una fila sino será None
    bbdd_existe = cur.fetchone()
    
    # Si bbdd_existe es None, creamos la base de datos
    if not bbdd_existe:
        cur.execute(f"CREATE DATABASE {database_name};")
        print(f"Base de datos {database_name} creada con éxito")
    else:
        print(f"La base de datos ya existe")
        
    # Cerramos el cursor y la conexion
    cur.close()
    conn.close()


## Creacionde base de Tablas


In [8]:
conexion = establecer_conn("sanidad42", "admin", "postgres")

# creamos cada una de las tablas de la base de datos.
conexion.cursor().execute(sqc.query_creation_hospitales)
conexion.cursor().execute(sqc.query_creation_tipo_hosp)
conexion.cursor().execute(sqc.query_creation_gastos)
conexion.cursor().execute(sqc.query_creation_ingresos)